# SICP-3.2.4 Digital circuit simulator

## Event driven simulation
helper functions to validate signals, and setup the global environment

In [ ]:
; print function not working well in this jupyter-playbook, so we define a new
; print which bound to the 'display' procedure
;(define (print x)(display x))

(define (logical-not s) 
  (cond ((= s 0) 1) 
        ((= s 1) 0) 
        (else (error "Invalid signal" s))))

(define (logical-and s1 s2)
  (cond((not(or (= s1 0) (= s1 1))) error "Invalid signal" s1 )
       ((not(or (= s2 0) (= s2 1))) error "Invalid signal" s2 )
        (else (and s1 s2))))    

(define (logical-or s1 s2)
  (cond((not(or (= s1 0) (= s1 1))) error "Invalid signal" s1 )
       ((not(or (= s2 0) (= s2 1))) error "Invalid signal" s2 )
        (else (or s1 s2))))    

; define proc to call all procs in a given list
(define (call-each procedures)
  (if (null? procedures) 
    'done 
    (begin ((car procedures)) 
            (call-each (cdr procedures)))))


In [ ]:
; wire object holds a signal
; when signal changed - all actions get called

(define (make-wire) 
  (let ((signal-value 0) (action-procedures '()))  ; init to 0 with empty actions list
    (define (set-my-signal! new-value) 
      (if (not (= signal-value new-value)) 
          (begin (set! signal-value new-value) 
                 (call-each action-procedures)) 
          'done)) 
    (define (accept-action-procedure! proc)        ; add action to list
      (set! action-procedures (cons proc action-procedures)) 
      (proc)) 
    (define (dispatch m) 
      (cond ((eq? m 'get-signal) signal-value) 
            ((eq? m 'set-signal!) set-my-signal!) 
            ((eq? m 'add-action!) accept-action-procedure!) 
            (else (error "Unknown operation: WIRE" m)))) 
    dispatch))


In [ ]:
; create 6 wires for implementing half-adder
(define a (make-wire)) 
(define b (make-wire)) 
(define c (make-wire)) 
(define d (make-wire)) 
(define e (make-wire)) 
(define s (make-wire))


In [ ]:
; syntactic sugar. call object method as a function call [ie: obj.do(); == do(obj); ]
(define (get-signal wire) (wire 'get-signal)) 
(define (set-signal! wire new-value) ((wire 'set-signal!) new-value)) 
(define (add-action! wire action-procedure) ((wire 'add-action!) action-procedure))

![title](img/logic-gates.png)

In [ ]:
; inverter accepts 2 wires, and creates invert-input function
; that been added to the wire actions-list (connect to the wire)
(define (inverter input output) 
  (define (invert-input) 
    (let ((new-value (logical-not (get-signal input))))
      (after-delay inverter-delay 
        (lambda () 
          (set-signal! output new-value))))) 
  (add-action! input invert-input) 'ok) 

; and-gate accepts 3 wires, create action and bind to both input wires
(define (and-gate a1 a2 output)
  (define (and-action-procedure)
    (let ((new-value (logical-and (get-signal a1) (get-signal a2))))
      (after-delay and-gate-delay 
        (lambda () 
          (set-signal! output new-value)))))
  (add-action! a1 and-action-procedure)
  (add-action! a1 and-action-procedure)
  'ok)

; or-gate accepts 3 wires, create action and bind to both input wires
(define (or-gate o1 o2 output)
  (define (or-action-procedure)
    (let ((new-value (logical-or (get-signal o1) (get-signal o2))))
      (after-delay or-gate-delay 
        (lambda () 
          (set-signal! output new-value)))))
  (add-action! o1 or-action-procedure)
  (add-action! o2 or-action-procedure)
  'ok)


![title](img/half-adder.png)

In [ ]:
; construction of gates, by providing wires as args
(or-gate a b d)
(and-gate a b c)
(inverter c e)
(and-gate d e s)


In [ ]:
(define (half-adder a b s c)
  (let ((d (make-wire)) (e (make-wire)))
    (or-gate a b d)
    (and-gate a b c)
    (inverter c e)
    (and-gate d e s) 'ok))

![title](img/full-adder.png)

In [ ]:
(define (full-adder a b c-in sum c-out) 
  (let ((s (make-wire)) (c1 (make-wire)) (c2 (make-wire))) 
    (half-adder b c-in s c1) 
    (half-adder a s sum c2) 
    (or-gate c1 c2 c-out) 'ok))


## Agenda
Agenda is a data structure, that contains a schedule of things to do.
```
(make-agenda)
(empty-agenda? ⟨agenda⟩)
(first-agenda-item ⟨agenda⟩)
(remove-first-agenda-item! ⟨agenda⟩)
(add-to-agenda! ⟨time⟩ ⟨action⟩ ⟨agenda⟩)
(current-time ⟨agenda⟩)

```

In [ ]:
; schedule an action in the agenda with the provided delay time
(define (after-delay delay action) 
  (add-to-agenda! (+ (current-time the-agenda) delay) 
                  action
                  the-agenda ))

; recursive function executes first item and remove it, then call again
(define (propogate)
  (if (empty-agenda? the-agenda) 
      'done 
      (let ((first-item (first-agenda-item the-agenda))) 
            (first-item)
            (remove-first-agenda-item! the-agenda)
            (propogate))))

; prob is attached to a wire and register a callback to invoke when its state changed 
(define (prob name wire)
 (add-action! wire 
   (lambda ()
     (newline)
     (display name) (display " ")
     (display (current-time the-agenda))
     (display " new-value: ")
     (display (get-signal wire)))))

In [ ]:
(define (prob name wire)
 (add-action! wire 
   (lambda ()
     (newline)
     (display name) (display " ")
     (display (current-time the-agenda))
     (display " new-value: ")
     (display (get-signal wire)))))

In [ ]:
(define x (cons 1 2))
x

In [ ]:
(set-car! x 3)
x

# FULL WORKING IN https://scheme.cs61a.org/ INTERPRETER

In [ ]:
#!r5rs
;#lang racket

;(require rnrs/mutable-pairs-6)

(define (logical-not s) 
  (cond ((= s 0) 1) 
        ((= s 1) 0) 
        (else (error "Invalid signal" s))))

(define (logical-and s1 s2)
  (cond((not(or (= s1 0) (= s1 1))) error "Invalid signal" s1 )
       ((not(or (= s2 0) (= s2 1))) error "Invalid signal" s2 )
        (else (and s1 s2))))    

(define (logical-or s1 s2)
  (cond((not(or (= s1 0) (= s1 1))) error "Invalid signal" s1 )
       ((not(or (= s2 0) (= s2 1))) error "Invalid signal" s2 )
        (else (or s1 s2))))    

; define proc to call all procs in a given list
(define (call-each procedures)
  (if (null? procedures) 
    'done 
    (begin ((car procedures)) 
            (call-each (cdr procedures)))))

; wire object holds a signal
; when signal changed - all actions get called

(define (make-wire) 
  (let ((signal-value 0) (action-procedures '()))  ; init to 0 with empty actions list
    (define (set-my-signal! new-value) 
      (if (not (= signal-value new-value)) 
          (begin (set! signal-value new-value) 
                 (call-each action-procedures)) 
          'done)) 
    (define (accept-action-procedure! proc)        ; add action to list
      (set! action-procedures (cons proc action-procedures)) 
      (proc)) 
    (define (dispatch m) 
      (cond ((eq? m 'get-signal) signal-value) 
            ((eq? m 'set-signal!) set-my-signal!) 
            ((eq? m 'add-action!) accept-action-procedure!) 
            (else (error "Unknown operation: WIRE" m)))) 
    dispatch))

; create 6 wires for implementing half-adder
(define a (make-wire)) 
(define b (make-wire)) 
(define c (make-wire)) 
(define d (make-wire)) 
(define e (make-wire)) 
(define s (make-wire))

; syntactic sugar. call object method as a function call [ie: obj.do(); == do(obj); ]
(define (get-signal wire) (wire 'get-signal)) 
(define (set-signal! wire new-value) ((wire 'set-signal!) new-value)) 
(define (add-action! wire action-procedure) ((wire 'add-action!) action-procedure))

; inverter accepts 2 wires, and creates invert-input function
; that been added to the wire actions-list (connect to the wire)
(define (inverter input output) 
  (define (invert-input) 
    (let ((new-value (logical-not (get-signal input))))
      (after-delay inverter-delay 
        (lambda () 
          (set-signal! output new-value))))) 
  (add-action! input invert-input) 'ok) 

; and-gate accepts 3 wires, create action and bind to both input wires
(define (and-gate a1 a2 output)
  (define (and-action-procedure)
    (let ((new-value (logical-and (get-signal a1) (get-signal a2))))
      (after-delay and-gate-delay 
        (lambda () 
          (set-signal! output new-value)))))
  (add-action! a1 and-action-procedure)
  (add-action! a1 and-action-procedure)
  'ok)

; or-gate accepts 3 wires, create action and bind to both input wires
(define (or-gate o1 o2 output)
  (define (or-action-procedure)
    (let ((new-value (logical-or (get-signal o1) (get-signal o2))))
      (after-delay or-gate-delay 
        (lambda () 
          (set-signal! output new-value)))))
  (add-action! o1 or-action-procedure)
  (add-action! o2 or-action-procedure)
  'ok)

(define (half-adder a b s c)
  (let ((d (make-wire)) (e (make-wire)))
    (or-gate a b d)
    (and-gate a b c)
    (inverter c e)
    (and-gate d e s) 'ok))

(define (full-adder a b c-in sum c-out) 
  (let ((s (make-wire)) (c1 (make-wire)) (c2 (make-wire))) 
    (half-adder b c-in s c1) 
    (half-adder a s sum c2) 
    (or-gate c1 c2 c-out) 'ok))

; schedule an action in the agenda with the provided delay time
(define (after-delay delay action) 
  (add-to-agenda! (+ (current-time the-agenda) delay) 
                  action
                  the-agenda ))

; recursive function executes first item and remove it, then call again
(define (propagate)
  (if (empty-agenda? the-agenda) 
      'done 
      (let ((first-item (first-agenda-item the-agenda))) 
            (first-item)
            (remove-first-agenda-item! the-agenda)
            (propagate))))

; probe is attached to a wire and register a callback to invoke when its state changed 
(define (probe name wire)
 (add-action! wire 
   (lambda ()
     (newline)
     (display name) (display " ")
     (display (current-time the-agenda))
     (display " new-value: ")
     (display (get-signal wire)))))
(define (make-agenda) (list 0))
(define (current-time agenda) (car agenda))
(define (set-current-time! agenda time) (set-car! agenda time))
(define (segments agenda) (cdr agenda))
(define (set-segments! agenda segments) (set-cdr! agenda segments))
(define (first-segment agenda) (car (segments agenda)))
(define (rest-segments agenda) (cdr (segments agenda)))
(define (empty-agenda? agenda) (null? (segments agenda)))

(define (make-time-segment time queue)
  (cons time queue)) 
(define (segment-time s) (car s))
(define (segment-queue s) (cdr s))

;(define (first-agenda-item agenda)
;  (if (empty-agenda?)
;      (error "Agenda is empty: FIRST-AGENDA-ITEM") 
;      (car agenda)))

(define (first-agenda-item agenda)
  (if (empty-agenda?)
      (error "Agenda is empty: FIRST-AGENDA-ITEM") 
    (let ((first-seg (first-segment agenda)))
         (set-current-time! agenda (segment-time first-seg))
         (front-queue (segment-queue first-seg)))))


(define (add-to-agenda! time action agenda)
  (define (belongs-before? segments)
    (or (null? segments) (< time (segment-time (car segments)))))
  (define (make-new-time-segment time action)
    (let ((q (make-queue)))
          (insert-queue! q action)
          (make-time-segment time q)))
  (define (add-to-segments! segments)
    (if (= (segment-time (car segments)) time)
          (insert-queue! (segment-queue (car segments)) action)
        (let ((rest (cdr segments)))
          (if (belongs-before? rest)
              (set-cdr! segments (cons (make-new-time-segment time action) (cdr segments)))
          (add-to-segments! rest)))))
  (let ((segments (segments agenda)))
    (if (belongs-before? segments)
        (set-segments! agenda (cons (make-new-time-segment time action) segments))
    (add-to-segments! segments))))
(define (remove-first-agenda-item! agenda)
  (let ((q (segment-queue (first-segment agenda))))
    (delete-queue! q)
    (if (empty-queue? q)
        (set-segments! agenda (rest-segments agenda))
        'nop)))


; queues
(define (make-queue) (cons '() '()))
(define (front-queue queue)
  (if (empty-queue? queue)
      (error "FRONT called with an empty queue"
             queue)
      (car (front-ptr queue))))
(define (front-ptr queue) (car queue))
(define (rear-ptr queue) (cdr queue))
(define (set-front-ptr! queue item)
(set-car! queue item))
(define (set-rear-ptr! queue item)
  (set-cdr! queue item))
(define (empty-queue? queue)
  (null? (front-ptr queue)))
(define (insert-queue! queue item)
  (let ((new-pair (cons item '())))
    (cond ((empty-queue? queue)
           (set-front-ptr! queue new-pair)
           (set-rear-ptr! queue new-pair)
           queue)
        (else
          (set-cdr! (rear-ptr queue) new-pair)
          (set-rear-ptr! queue new-pair)
          queue))))
(define (delete-queue! queue)
  (cond ((empty-queue? queue)
         (error "DELETE! called with an empty queue" queue))
      (else (set-front-ptr! queue (cdr (front-ptr queue)))
         queue)))

; setup simulation
(define the-agenda (make-agenda))
(define inverter-delay 2)
(define and-gate-delay 3)
(define or-gate-delay 5)

(define input-1 (make-wire))
(define input-2 (make-wire))
(define sum (make-wire))
(define carry (make-wire))

; running manual simulation
(probe 'sum sum)
(probe 'carry carry)
(half-adder input-1 input-2 sum carry)
(set-signal! input-1 1)